# ParkingSpacePicker 

In [1]:
#Import Libraries
import cv2
import pickle

In [2]:
# Global variables
width, height = 107, 48
posList = []

In [3]:
# Function to save positions to a file using pickle
def save_positions():
    with open('CarParkPos', 'wb') as f:
        pickle.dump(posList, f)


In [4]:
# Function to load positions from a file
def load_positions():
    global posList
    try:
        with open('CarParkPos', 'rb') as f:
            posList = pickle.load(f)
    except:
        posList = []

In [5]:
# Function to handle mouse clicks
def mouseClick(events, x, y, flags, params):
    global posList
    if events == cv2.EVENT_LBUTTONDOWN: # Left mouse button clicked
        posList.append((x, y)) # Append the clicked position to the list
    if events == cv2.EVENT_RBUTTONDOWN:# Right mouse button clicked
        for i, pos in enumerate(posList): # Iterate through the positions
            x1, y1 = pos
            # Check if the clicked position is within any existing rectangle
            if x1 < x < x1 + width and y1 < y < y1 + height:
                posList.pop(i)  # If yes, remove that position
        save_positions() # Save positions after modification


In [ ]:

def main():
    load_positions()
    while True:
        img = cv2.imread('carParkImg.png')
        for pos in posList:
            cv2.rectangle(img, pos, (pos[0] + width, pos[1] + height), (255, 0, 255), 2)

        cv2.imshow("Image", img)
        cv2.setMouseCallback("Image", mouseClick)
        key = cv2.waitKey(1)
        if key == ord('q'):  # Press 'q' to quit
            save_positions()
            break

if __name__ == "__main__":
    main()

# Main

In [1]:
import cv2
import pickle
import cvzone  # Assuming this is an external library for drawing
import numpy as np

In [2]:
# Video feed
cap = cv2.VideoCapture('carPark.mp4')  # Open video file


In [3]:
# Load parking space positions from a pickle file
with open('CarParkPos', 'rb') as f:
    posList = pickle.load(f)

In [4]:
# Define width and height of the parking spaces
width, height = 107, 48


In [5]:
# Function to check the occupancy of parking spaces
def checkParkingSpace(imgPro):
    spaceCounter = 0  # Counter for free parking spaces

    # Iterate through each parking space position
    for pos in posList:
        x, y = pos

        # Crop the image to the region of interest (a parking space)
        imgCrop = imgPro[y:y + height, x:x + width]
        
        # Count non-zero pixels in the cropped image
        count = cv2.countNonZero(imgCrop)
        
        # Determine color and thickness for drawing rectangle based on occupancy
        if count < 900:  # Assuming a threshold for considering a parking space occupied
            color = (0, 255, 0)  # Green color for free space
            thickness = 5
            spaceCounter += 1  # Increment free space counter
        else:
            color = (0, 0, 255)  # Red color for occupied space
            thickness = 2

        # Draw rectangle around the parking space
        cv2.rectangle(img, pos, (pos[0] + width, pos[1] + height), color, thickness)

        # Put text showing the count of non-zero pixels (some measure of occupancy)
        cvzone.putTextRect(img, str(count), (x, y + height - 3), scale=1,
                           thickness=2, offset=0, colorR=color)

    # Put text showing the total number of free parking spaces
    cvzone.putTextRect(img, f'Free: {spaceCounter}/{len(posList)}', (100, 50), scale=3,
                           thickness=5, offset=20, colorR=(0,200,0))


In [ ]:
# Main loop to process each frame of the video
while True:
    # Check if we have reached the end of the video, if so, restart from the beginning
    if cap.get(cv2.CAP_PROP_POS_FRAMES) == cap.get(cv2.CAP_PROP_FRAME_COUNT):
        cap.set(cv2.CAP_PROP_POS_FRAMES, 0)

    # Read a frame from the video
    success, img = cap.read()

    # Convert the frame to grayscale
    imgGray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur to the grayscale image
    imgBlur = cv2.GaussianBlur(imgGray, (3, 3), 1)

    # Apply adaptive thresholding to get a binary image
    imgThreshold = cv2.adaptiveThreshold(imgBlur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                         cv2.THRESH_BINARY_INV, 25, 16)

    # Apply median blur to the thresholded image
    imgMedian = cv2.medianBlur(imgThreshold, 5)

    # Apply dilation to the median blurred image
    kernel = np.ones((3, 3), np.uint8)
    imgDilate = cv2.dilate(imgMedian, kernel, iterations=1)

    # Check parking space occupancy using the processed image
    checkParkingSpace(imgDilate)

    # Display the processed image with parking space occupancy information
    cv2.imshow("Image", img)

    # Wait for a key press, with a delay of 10 milliseconds
    cv2.waitKey(10)